In [ ]:
import numpy as np

# Dim 1

In [ ]:
expl = np.array([False, True, True, False, False])

In [ ]:
splits = []
last_val = None
for ind, val in enumerate(expl):
    if val != last_val:
        splits.append([ind])
        last_val = val
    else:
        splits[-1].append(ind)
splits

In [ ]:
splits = []
last_val = None
for ind, val in enumerate(expl):
    if val != last_val:
        splits.append([ind])
        last_val = val
    elif val:
        splits[-1].append(ind)
    else:
        splits.append([ind])
splits

In [ ]:
lenghts = [[len(split)] * len(split) for split in splits]
lenghts = [ele for sub in lenghts for ele in sub]
lenghts

In [ ]:
def add(bol, val):
    return val if bol else 0

from numpy import vectorize

add_vecs = vectorize(add)

def consecutive(expl):
    splits = []
    last_val = None
    for ind, val in enumerate(expl):
        if val != last_val:
            splits.append([ind])
            last_val = val
        elif val:
            splits[-1].append(ind)
        else:
            splits.append([ind])
    lenghts = [[len(split)] * len(split) for split in splits]
    lengths = [ele for sub in lenghts for ele in sub]
    return add_vecs(expl, np.array(lengths))

consecutive(expl)

# Dim 2

In [ ]:
from itertools import product

In [ ]:
expl = np.array([
    [False, True, True, False, False],
    [True, True, True, True, False],
    [False, True, False, False, False],
])
expl

In [ ]:
np.array([consecutive(row) for row in expl])

In [ ]:
np.array([consecutive(col) for col in expl.T]).T

In [ ]:
n_rows, n_cols = expl.shape

counts = {size: 0 for size in range(n_rows * n_cols + 1)}
for left, right in product(range(n_cols + 1), repeat=2):
    for bottom, top in product(range(n_rows + 1), repeat=2):
        if left < right and bottom < top:
            size = expl[bottom:top, left:right].size
            counts[size] += 1
counts

In [ ]:
n_rows, n_cols = expl.shape

rects = []
areas = []
for left, right in product(range(n_cols + 1), repeat=2):
    for bottom, top in product(range(n_rows + 1), repeat=2):
        if left < right and bottom < top:
            sub = expl[bottom:top, left:right]
            if (~sub).sum() == 0:
                rects.append([(bottom, top), (left, right)])
                areas.append(sub.size)

areas_ = [len(list(product(range(*rect[0]), range(*rect[1])))) for rect in rects]
assert areas_ == areas

pts = product(range(n_rows), range(n_cols))
max_areas = {pt: [] for pt in pts}
for rect, area in zip(rects, areas):
    pts_ = list(product(range(*rect[0]), range(*rect[1])))
    for pt in pts_:
        max_areas[pt].append(area)

max_areas = {pt: (max(lst) if lst else 0) for pt, lst in max_areas.items()}
np.array(list(max_areas.values())).reshape(n_rows, n_cols)

In [ ]:
def max_areas(expl):
    n_rows, n_cols = expl.shape

    # list all rectangles of only ones
    rects = []
    for left, right in product(range(n_cols + 1), repeat=2):
        for bottom, top in product(range(n_rows + 1), repeat=2):
            if left < right and bottom < top:
                sub = expl[bottom:top, left:right]
                if (~sub).sum() == 0:
                    rects.append([(bottom, top), (left, right)])

    # list all points belonging to the previous rectangles
    pts = product(range(n_rows), range(n_cols))
    max_areas = {pt: [] for pt in pts}
    for rect in rects:
        pts_ = list(product(range(*rect[0]), range(*rect[1])))
        area = len(pts_)
        for pt in pts_:
            max_areas[pt].append(area)

    # take max areas
    return (
        np.array([
            (max(lst) if lst else 0)
            for lst in max_areas.values()])
        .reshape(n_rows, n_cols)
    )

In [ ]:
expl

In [ ]:
max_areas(expl)

In [ ]:
def max_areas(expl):
    dim_0, dim_1 = expl.shape

    # list all rectangles of only ones
    rects = []
    for pair_0 in product(range(dim_0 + 1), repeat=2):
        for pair_1 in product(range(dim_1 + 1), repeat=2):
            if pair_0[0] < pair_0[1] and pair_1[0] < pair_1[1]:
                sub = expl[slice(pair_0[0], pair_0[1]), slice(pair_1[0], pair_1[1])]
                if (~sub).sum() == 0:
                    rects.append([pair_0, pair_1])

    # list all points belonging to the previous rectangles
    pts = product(range(dim_0), range(dim_1))
    max_areas = {pt: [] for pt in pts}
    for pair_0, pair_1 in rects:
        pts_ = list(product(range(*pair_0), range(*pair_1)))
        area = len(pts_)
        for pt in pts_:
            max_areas[pt].append(area)

    # take max areas
    return (
        np.array([
            (max(lst) if lst else 0)
            for lst in max_areas.values()])
        .reshape(dim_0, dim_1)
    )

max_areas(expl)

# Dim n

In [ ]:
dims = expl.shape
# n_dims = len(dims)
# sizes = []

prods = [product(range(dim + 1), repeat=2) for dim in dims]
for pairs in product(*prods):
    if all(pair[0] < pair[1] for pair in pairs):
        slices = [np.arange(pair[0], pair[1]) for pair in pairs]
        sub = expl[np.ix_(*slices)]
        if (~sub).sum() == 0:
            rects.append(pairs)

pts = product(*[range(dim) for dim in dims])
max_areas = {pt: [] for pt in pts}
for pairs in rects:
    pts_ = list(product(*[range(*pair) for pair in pairs]))
    area = len(pts_)
    for pt in pts_:
        max_areas[pt].append(area)

(
        np.array([
            (max(lst) if lst else 0)
            for lst in max_areas.values()])
        .reshape(dims)
    )

In [ ]:
def max_volumes(expl):
    dims = expl.shape
    pts = product(*[range(dim) for dim in dims])
    volumes = {pt: [] for pt in pts}

    bounds = [product(range(dim + 1), repeat=2) for dim in dims]
    all_pairs = [
        pairs
        for pairs in product(*bounds)
        if all(pair[0] < pair[1] for pair in pairs)
    ]
    for pairs in all_pairs:
        slices = [np.arange(*pair) for pair in pairs]
        pts_in_slice = list(product(*slices))
        sub = expl[np.ix_(*slices)]
        assert (volume := sub.size) == len(pts_in_slice)
        if (~sub).sum() == 0:
            for pt_in_slice in pts_in_slice:
                volumes[pt_in_slice].append(volume)

    return np.array([
        (max(vols) if vols else 0)
        for vols in volumes.values()]).reshape(dims)

In [ ]:
expl = np.array([False, True, True, False, False])
max_volumes(expl)

In [ ]:
expl = np.array([
    [False, True, True, False, False],
    [True, True, True, True, False],
    [False, True, False, False, False],
])
max_volumes(expl)

In [ ]:
expl = np.random.rand(2, 5, 6) < 0.5
expl

In [ ]:
%%time
max_volumes(expl)

In [ ]:
from itertools import product
from math import prod
import numpy as np


def update_inner_volumes(inner_volumes, expl, bounds):
    slices = [np.arange(*bound) for bound in bounds]
    sub = expl[np.ix_(*slices)]
    if (~sub).sum() == 0:
        inner_vol = prod((x - 1) for x in sub.shape)
        for pt_ in product(*slices):
            inner_volumes[pt_].append(inner_vol)


def max_inner_volumes(expl):
    dims = expl.shape
    pts = product(*[range(dim) for dim in dims])
    inner_volumes = {pt: [] for pt in pts}

    lst_bounds = [product(range(dim + 1), repeat=2) for dim in dims]
    _ = [
        update_inner_volumes(inner_volumes, expl, bounds)
        for bounds in product(*lst_bounds)
        if all(bound[0] < bound[1] for bound in bounds)
    ]

    return np.array([
        (max(vols) if vols else 0)
        for vols in inner_volumes.values()]).reshape(dims)


In [ ]:
expl = np.array([
    [False, True, True, False, False],
    [True, True, True, True, False],
    [False, True, False, False, False],
])
tes = max_inner_volumes(expl)
tes

In [ ]:
from multiprocessing import Pool
from itertools import product
from math import prod
import numpy as np


def loop_through_slices(expl, compute_inner_volume, processes=None):
    dims = expl.shape

    lst_bounds = [product(range(dim + 1), repeat=2) for dim in dims]
    pairs = [
        bounds for bounds in product(*lst_bounds)
        if all(bound[0] < bound[1] for bound in bounds)
    ]
    with Pool(processes) as pool:
        lst = list(pool.imap_unordered(compute_inner_volume, pairs))
    
    return lst

def pos_treat(expl, res):
    curated = [ele for ele in res if (ele is not None and ele[1] > 0)]
    dims = expl.shape
    pts = product(*[range(dim) for dim in dims])
    inner_volumes = {pt: [] for pt in pts}

    for pts_, vol in curated:
        for pt_ in pts_:
            inner_volumes[pt_].append(vol)

    return np.array([
        (max(vols) if vols else 0)
        for vols in inner_volumes.values()]).reshape(dims)

In [ ]:
%%time

expl = np.random.rand(20, 20, 20) < 0.8

def compute_inner_volume(bounds):
    res = None
    slices = [np.arange(*bound) for bound in bounds]
    sub = expl[np.ix_(*slices)]
    if (~sub).sum() == 0:
        inner_vol = prod((x - 1) for x in sub.shape)
        res = (product(*slices), inner_vol)
    return res

In [ ]:
%%time
res = loop_through_slices(expl, compute_inner_volume)

In [ ]:
%%time
volumes = pos_treat(expl, res)
volumes.max()